In [55]:
import pandas as pd
import numpy as np

In [56]:
#import dataset
original_election_data = pd.read_csv("election_train.csv")
demographic_data = pd.read_csv("demographics_train.csv")
original_election_data
#demographic_data

,Year,State,County,Office,Party,Votes
0,2018,AZ,Apache County,US Senator,Democratic,16298.0
1,2018,AZ,Apache County,US Senator,Republican,7810.0
2,2018,AZ,Cochise County,US Senator,Democratic,17383.0
3,2018,AZ,Cochise County,US Senator,Republican,26929.0
4,2018,AZ,Coconino County,US Senator,Democratic,34240.0
5,2018,AZ,Coconino County,US Senator,Republican,19249.0
6,2018,AZ,Gila County,US Senator,Democratic,7643.0
7,2018,AZ,Gila County,US Senator,Republican,12180.0
8,2018,AZ,Graham County,US Senator,Democratic,3368.0
9,2018,AZ,Graham County,US Senator,Republican,6870.0


In [57]:
#Task 1 : Reshape from long format to wide format
election_data = pd.pivot_table(original_election_data, index=['Year','State','County', 'Office'], columns=['Party'], values=['Votes'], aggfunc=np.sum).reset_index()
election_data.columns = election_data.columns.droplevel(1)
election_data.columns = ["Year", "State", "County", "Office", "Democratic Votes", "Republican Votes"]
election_data.count()
#election_data.head()
#election_data['County'].count()Z
#demographic_data['County'].count()
#new_election_data['Votes']['Democratic']

Year                1205
State               1205
County              1205
Office              1205
Democratic Votes    1205
Republican Votes    1205
dtype: int64

In [58]:
#demographic_data['County'].loc[demographic_data['County']=="James City"] = "James City County"
#demographic_data[demographic_data['County'] == "James City County"]

In [59]:
#election_data['County'].replace({'County', ''}, inplace=True, regex=False )
election_data['County'] = election_data['County'].str.replace('County', '')
election_data['County'] = election_data['County'].str.strip()
election_data['State'] = election_data['State'].str.strip()
#election_data
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
states2 = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinios': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Neveda': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhoda Island': 'RI',
    'South Carolina': 'SC',
    'South Dakoda': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
election_data = election_data.replace({"State":states})
demographic_data['State'] = demographic_data['State'].str.strip()
demographic_data['County'] = demographic_data['County'].str.strip()

demographic_data['State'] = demographic_data['State'].str.lower()
election_data['State'] = election_data['State'].str.lower()
election_data['County'] = election_data['County'].str.lower()
demographic_data['County'] = demographic_data['County'].str.lower()
election_data
#demographic_data
#election_data.count()

,Year,State,County,Office,Democratic Votes,Republican Votes
0,2018,arizona,apache,US Senator,16298.0,7810.0
1,2018,arizona,cochise,US Senator,17383.0,26929.0
2,2018,arizona,coconino,US Senator,34240.0,19249.0
3,2018,arizona,gila,US Senator,7643.0,12180.0
4,2018,arizona,graham,US Senator,3368.0,6870.0
5,2018,arizona,la paz,US Senator,1609.0,3265.0
6,2018,arizona,maricopa,US Senator,732671.0,672505.0
7,2018,arizona,mohave,US Senator,19214.0,50209.0
8,2018,arizona,navajo,US Senator,16624.0,18767.0
9,2018,arizona,pima,US Senator,221242.0,160550.0


In [60]:
#Task 2
merged_data = pd.merge(election_data, demographic_data, how='inner', on=['State','County'])
#merged_data[merged_data['County']==""]
merged_data

,Year,State,County,Office,Democratic Votes,Republican Votes,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino",...,Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,2018,arizona,apache,US Senator,16298.0,7810.0,4001,72346,0,18.571863,...,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076
1,2018,arizona,cochise,US Senator,17383.0,26929.0,4003,128177,92915,56.299492,...,34.403208,11.458374,49.069646,37.902276,19.756275,45383,8.567108,13.409171,76.837055,36.301067
2,2018,arizona,coconino,US Senator,34240.0,19249.0,4005,138064,104265,54.619597,...,13.711033,4.825298,50.581614,48.946141,10.873943,51106,8.238305,11.085381,65.791439,31.466066
3,2018,arizona,gila,US Senator,7643.0,12180.0,4007,53179,0,63.222325,...,18.548675,4.249798,50.296170,32.238290,26.397638,40593,12.129932,15.729958,82.262624,41.062000
4,2018,arizona,graham,US Senator,3368.0,6870.0,4009,37529,0,51.461536,...,32.097844,4.385942,46.313518,46.393456,12.315809,47422,14.424104,14.580797,86.675944,46.437399
5,2018,arizona,la paz,US Senator,1609.0,3265.0,4012,20304,15245,58.884949,...,26.182033,11.372143,48.946020,28.073286,36.056935,36321,10.599013,24.842215,89.563407,56.327786
6,2018,arizona,maricopa,US Senator,732671.0,672505.0,4013,4088549,2723565,56.918114,...,30.286833,14.729333,50.549278,41.886620,13.837843,55676,6.808454,13.051927,69.031137,2.363800
7,2018,arizona,mohave,US Senator,19214.0,50209.0,4015,203629,0,78.252606,...,15.708470,6.969047,49.676618,30.485835,26.858650,39856,11.680953,16.145850,88.121178,22.963644
8,2018,arizona,navajo,US Senator,16624.0,18767.0,4017,108209,76280,41.927196,...,11.049913,2.914730,49.846131,43.243168,15.745456,36868,18.525791,18.494087,85.507970,54.138242
9,2018,arizona,pima,US Senator,221242.0,160550.0,4019,1003338,0,53.271579,...,36.105978,12.903428,50.807405,40.087388,17.801778,46764,9.214114,12.252238,69.199391,7.523491


In [64]:
#Task 5
merged_data['Party'] = np.where(merged_data['Democratic Votes'] > merged_data['Republican Votes'], 1, 0)
#merged_data[['Democratic Votes', 'Republican Votes', 'Party']]

,Democratic Votes,Republican Votes,Party
0,16298.0,7810.0,1
1,17383.0,26929.0,0
2,34240.0,19249.0,1
3,7643.0,12180.0,0
4,3368.0,6870.0,0
5,1609.0,3265.0,0
6,732671.0,672505.0,1
7,19214.0,50209.0,0
8,16624.0,18767.0,0
9,221242.0,160550.0,1


In [70]:
#Task 6(TODO)
demo_mean = merged_data.groupby(['Party']).get_group(1).mean()
repub_mean = merged_data.groupby(['Party']).get_group(0).mean()
demo_mean['Total Population'] > repub_mean['Total Population']

True

In [72]:
#Task 7(TODO)
demo_mean = merged_data.groupby(['Party']).get_group(1).mean()
repub_mean = merged_data.groupby(['Party']).get_group(0).mean()
demo_mean['Median Household Income'] > repub_mean['Median Household Income']

True

In [ ]:
#Task 8()